模块导入,是文件和文件之间的调用
想再新建一个cal   文件已经存在,是不能新建的
名字相同,但能再不同文件夹路径下,但我又需要用到两个同名py内不同的函数,变量,类  怎么办?

package和directory的区别是?
directory:一般非py文件放文件夹
package:全是py文件  区别在 package里有一__init__.py文件  ,在directory文件夹下加__init__.py  那么directory也能变package
在不同包里的py文件,同名也不会有冲突,命名最好不要带数字和点
一个包里 是多个模块,包的概念比模块又大了

项目--->包---->模块---->类---->函数---->变量

那这个__init__.py文件有什么作用呢?

使用包中  模块中的User类  怎么用：如下

In [10]:
import importlib
import sys
def reload_module(module_name):
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        __import__(module_name)

from user52test import models
reload_module('user52test.models')  #重新加载包里模块时候,要用点 调用
u=models.User('admin','123456')
u.show()

Username: admin
Password: 123456


上面这样确实很别扭  虽然确实能调用 从user包中找到models 再调用其中User类  更简洁的，直接调包的过程，用包名.模块名  import  User类

但是由此可见  不管怎样  函数或者类名 最好是别相同，相同应该也有调用顺序的问题。包需要不同，模块名可以相同，但模块内的类名，变量，函数 不要相同。

In [18]:
import importlib
import sys
def reload_module(module_name):
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        __import__(module_name)
from user52test.models import User
reload_module('user52test.models')  #重新加载包里模块时候,要用点 调用
u = User('admin', '123456')
u.show()   #用了User的show方法  
from article52test.models import Article
reload_module('article52test.models')
a=Article('个人总结','zz')
a.show()    #用了Article的show方法  所以二者不会串 即使方法名同名。

Username: admin
Password: 123456
文章作者:个人总结文章名字:zz


如果在两个模块中定义了相同的类名，并且这两个类都有各自不同的 show 方法，那么在使用这两个类创建对象时，仍然不会串。因为每个模块的命名空间是独立的，即使类名相同，它们也会根据所属模块来区分, 只要用as  设置一个别名:
from user52test.models import MyClass as UserClass
from article52test.models import MyClass as ArticleClass

如果不用别名:
如果你在两个不同的模块中定义了相同的类名，并且在导入时没有使用别名，那么后导入的类会覆盖先前导入的类。这会导致混淆和潜在的错误，因为两个类名相同，但它们的定义和行为不同。

实际运行会根据所属的类调用方法,不会混淆,除非类名都相同.
每个模块有自己的命名空间，即使类名相同，它们也不会冲突。
别名：在导入时使用别名（例如 MyClass as UserClass 和 MyClass as ArticleClass）可以帮助区分相同名称的类。
方法绑定：对象会根据其所属的类调用相应的方法，而不会混淆到其他类的方法。

对导入包的要求:1.由于路径,需要包的路径和当前运行的文件要同级,否则就要另外配置路径
2.__init__.py 文件的存在使得 Python 将包含该文件的目录视为一个包。
即使 __init__.py 文件是空的，它也必须存在，以便 Python 能够正确地识别和导入包。
下面是伪代码,不能运行的



In [19]:
# user52test/models.py
class MyClass:
    def __init__(self, username, password):
        self.username = username
        self.password = password

    def show(self):
        print(f"User: {self.username}, Password: {self.password}")

# article52test/models.py
class MyClass:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def show(self):
        print(f"Article: {self.title}, Author: {self.author}")

from user52test.models import MyClass as UserClass
reload_module('user52test.models')
u = UserClass('admin', '123456')
u.show()  # 输出: User: admin, Password: 123456

from article52test.models import MyClass as ArticleClass
reload_module('article52test.models')
a = ArticleClass('个人总结', 'zz')
a.show()  # 输出: Article: 个人总结, Author: zz



ImportError: cannot import name 'MyClass' from 'user52test.models' (D:\Project\PycharmProject\Python_Beginner\user52test\models.py)

绝对导入和相对导入(以下两段都是不可以运行的):
模块之间调用,是可以用相对导入的,(但建议是绝对导入)如果主程序中有__name__==__main__的 ,这个py文件称为顶级脚本,顶级脚本导入 只能用绝对导入.   点 .models  的点表示从和本文件所处目录相同的目录下导入,即相对路径.

总结三种方式
from 包 import 模块
from 包.模块 import 类/函数/变量
from 包.模块 import *   (在那个模块里添加__all__)=['User']   __all__仅在*时候有用,两个是配合使用的


In [ ]:
#相对导入:
# main.py
from .user52test.models import MyClass as UserClass  # 这将导致错误
from .article52test.models import MyClass as ArticleClass  # 这也将导致错误

if __name__ == "__main__":
    u = UserClass('admin', '123456')
    u.show()

    a = ArticleClass('个人总结', 'zz')
    a.show()


In [ ]:
# main.py   绝对导入
from user52test.models import MyClass as UserClass
from article52test.models import MyClass as ArticleClass

if __name__ == "__main__":
    u = UserClass('admin', '123456')
    u.show()

    a = ArticleClass('个人总结', 'zz')
    a.show()
